In [1]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

In [2]:
data_root_path = "../data/Challenge/"
embedding_path = None
KG_root_path = None
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title','vert','entity'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
             'max_entity_num':5}

In [3]:
News = NewsContent(config)
TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
TestUsers = UserContent(News.news_index,config,'test.tsv',2)

19779
2472
2473


In [4]:
TrainUsers.session

[[['N9759966'],
  [31653],
  31653,
  ['N9759966-1'],
  ['N9774516-0',
   'N9771051-0',
   'N9770028-0',
   'N9775402-0',
   'N9774461-0',
   'N9759544-0',
   'N9773947-0',
   'N9142581-0',
   'N9775331-0',
   'N9775371-0']],
 [['N9778661'],
  [31711],
  31711,
  ['N9778661-1'],
  ['N9778669-0',
   'N9778736-0',
   'N9778623-0',
   'N9089120-0',
   'N9777492-0',
   'N9778718-0',
   'N9778657-0',
   'N9778682-0',
   'N9482970-0',
   'N9718262-0',
   'N9718298-0',
   'N9778728-0',
   'N9080070-0',
   'N9420172-0',
   'N9717914-0',
   'N9777397-0']],
 [['N9777856'],
  [31711],
  31711,
  ['N9777856-1'],
  ['N9778369-0',
   'N9778500-0',
   'N9778021-0',
   'N9778627-0',
   'N9778351-0',
   'N9778155-0',
   'N9778226-0',
   'N9777034-0',
   'N9778628-0',
   'N9778448-0']],
 [['N9776566'],
  [31685],
  31685,
  ['N9776566-1'],
  ['N9776715-0',
   'N9776406-0',
   'N9776071-0',
   'N9776808-0',
   'N9776246-0',
   'N9776497-0',
   'N9776046-0',
   'N9776855-0']],
 [['N9776553'],
  [31685],
 

In [23]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

19779
19869
2473
2472


In [ ]:
# title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)

In [ ]:
train_generator = TrainGenerator(News,TrainUsers,train_sess,train_user_id,train_buckets,train_label,32)
test_user_generator = UserGenerator(News,TestUsers,32)
val_user_generator = UserGenerator(News,ValidUsers,32)
news_generator = NewsGenerator(News,32)

In [ ]:
# for i in range(10):
#     model_config = {
#         'news_encoder':1,
#         'popularity_user_modeling':True,
#         'rel':True,
#         'ctr':True,
#         'content':True,
#         'rece_emb':True,
#         'activity':True
# 
#     }
# 
#     model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = create_pe_model(config,model_config,News,title_word_embedding_matrix,News.entity_embedding)
#     model.fit_generator(train_generator,epochs=2)
#     news_scoring = news_encoder.predict_generator(news_generator,verbose=True)
#     user_scoring = user_encoder.predict_generator(test_user_generator,verbose=True)
#     val_user_scoring = user_encoder.predict_generator(val_user_generator,verbose=True)
# 
# 
#     news_bias_vecs = bias_news_encoder.predict_generator(news_generator,verbose=True)
# 
#     if model_config['content'] and not model_config['rece_emb']:
#         bias_candidate_score = bias_content_scorer.predict(news_bias_vecs,batch_size=32,verbose=True)
#         bias_candidate_score = bias_candidate_score[:,0]
#     else:
#         bias_candidate_score = 0
# 
#     ctr_weight = scaler.get_weights()[0][0,0]
#     time_embedding_matrix = time_embedding_layer.get_weights()[0]
#     
#     predicted_activity_gates = activity_gater.predict(user_scoring,verbose=True)
#     predicted_activity_gates = predicted_activity_gates[:,0]
#     
#     val_predicted_activity_gates = activity_gater.predict(val_user_scoring,verbose=True)
#     val_predicted_activity_gates = val_predicted_activity_gates[:,0]
#     
#     rankings = news_ranking(model_config,ctr_weight,predicted_activity_gates,user_scoring,news_scoring,
#                                 bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                 News,test_impressions)
#     
#     val_rankings = news_ranking(model_config,ctr_weight,val_predicted_activity_gates,val_user_scoring,news_scoring,
#                                bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                News,val_impressions)
#     
#     performance = evaluate_performance(rankings,test_impressions)
#     val_performance = evaluate_performance(val_rankings,val_impressions)
# 
#     cold = []
#     for TOP_COLD_NUM in [0,1,3,5,]:
#         g = evaluate_cold_users(rankings,test_impressions,TestUsers.click,TOP_COLD_NUM)
#         cold.append(g)
#     diversity = []
#     for TOP_DIVERSITY_NUM in range(1,11):
#         div_top = evaluate_diversity_topic_all(TOP_DIVERSITY_NUM,rankings,test_impressions,News,TestUsers)
#         div_ilxd = evaluate_density_ILxD(TOP_DIVERSITY_NUM,rankings,test_impressions,news_scoring)
#         diversity.append([div_top,div_ilxd])
